In [3]:
import os

# pip3 install pillow
from PIL import Image, ImageFilter

import boto3
rek = boto3.client("rekognition")

def get_face_boxes(faces, source_size):
    return [
        (
            int(f['BoundingBox']['Left'] * source_size[0]),
            int(f['BoundingBox']['Top'] * source_size[1]),
            int((f['BoundingBox']['Left'] + f['BoundingBox']['Width']) * source_size[0]),
            int((f['BoundingBox']['Top'] + f['BoundingBox']['Height']) * source_size[1]),
            f['Pose']['Roll']
        )
        for f in faces
    ]


def blur_faces(image):
    im = Image.open(image)
    source_size = im.size[0], im.size[1]
    # move back to beginning of loaded buffer
    image.seek(0)
    resp = rek.detect_faces(Image={"Bytes": image.read()})
    faces = [face for face in resp['FaceDetails'] if face['Confidence'] > 90]
    
    for face_box in get_face_boxes(faces, source_size):
        print("Blurring Face")
        face_img = im.copy()
        blurred_face = face_img.crop(face_box[:4])
        blurred_face = blurred_face.filter(ImageFilter.GaussianBlur(20))
        im.paste(blurred_face, (face_box[0], face_box[1]))
    return im

In [5]:
pth = os.path.expanduser("~/Downloads/bieber.jpg")
with open(pth, 'rb') as f:
    img = blur_faces(f)

Blurring Face


In [8]:
import boto3
translate = boto3.client("translate")

In [9]:
help(translate.translate_text)

Help on method translate_text in module botocore.client:

translate_text(*args, **kwargs) method of botocore.client.Translate instance
    Translates input text from the source language to the target language. It is not necessary to use English (en) as either the source or the target language but not all language combinations are supported by Amazon Translate. For more information, see `Supported Language Pairs <http://docs.aws.amazon.com/translate/latest/dg/pairs.html>`__ .
    
     
    
     
    * Arabic (ar) 
     
    * Chinese (Simplified) (zh) 
     
    * Chinese (Traditional) (zh-TW) 
     
    * Czech (cs) 
     
    * Danish (da) 
     
    * Dutch (nl) 
     
    * English (en) 
     
    * Finnish (fi) 
     
    * French (fr) 
     
    * German (de) 
     
    * Hebrew (he) 
     
    * Indonesian (id) 
     
    * Italian (it) 
     
    * Japanese (ja) 
     
    * Korean (ko) 
     
    * Polish (pl) 
     
    * Portuguese (pt) 
     
    * Russian (ru) 
     
    